*Note: all those exercises should be done using python with pyccel openmp*

## Exercise 1 Hello World

1. Write an OpenMP program displaying the number of threads used for the execution and the rank of each of the threads.
3. Test the programs obtained with different numbers of threads for the parallel program.

**Output Example**
```shell
>> Python hello.py
Hello from the rank 2 thread
Hello from the rank 0 thread
Hello from the rank 3 thread
Hello from the rank 1 thread
Hello from the rank 4 thread
Parallel execution of hello_world with 4 threads
```
*Note that the output order maybe different*

In [ ]:
import numpy as np
import time
from pyccel.epyccel import epyccel
from pyccel.decorators import types

In [ ]:
%%file hello.py
from pyccel.stdlib.internal.openmp import omp_set_num_threads, omp_set_dynamic
from pyccel.decorators import types
from pyccel.stdlib.internal.openmp import omp_set_num_threads, omp_get_num_threads, omp_get_thread_num

@types('int')
def get_num_threads(n):
    #omp_set_dynamic(0)
    omp_set_num_threads(int(n))
    #$ omp parallel
    print("Hello from this$ omp end parallele rank ", int(omp_get_thread_num())," thread")
    result = omp_get_num_threads()
    #$ omp end parallel
    
    return result

if __name__ == "__main__" :
    x = get_num_threads(10)
    print("parallel execution of hello_world with ",x , "threads")

Overwriting hello.py


In [ ]:
!pyccel hello.py --language=c --openmp

In [ ]:
!./hello

Hello from this$ omp end parallele rank  1  thread
Hello from this$ omp end parallele rank  2  thread
Hello from this$ omp end parallele rank  3  thread
Hello from this$ omp end parallele rank  4  thread
Hello from this$ omp end parallele rank  5  thread
Hello from this$ omp end parallele rank  6  thread
Hello from this$ omp end parallele rank  7  thread
Hello from this$ omp end parallele rank  8  thread
Hello from this$ omp end parallele rank  9  thread
Hello from this$ omp end parallele rank  0  thread
parallel execution of hello_world with  10 threads


## Exercise 2 Matrix product

In [ ]:
def matrix_prod(A, B, C, N, M):
    
    for i in range( M ):
        for j in range( N ):
            A[ i, j ] = (i + 1) + (j + 1)

    
    for i in range( N ):
        for j in range( M ):
            B[ i, j ] = (i + 1) - (j + 1)

    
    for i in range( M ):
        for j in range( M ):
            C[ i, j ] = 0

    
    for i in range( M ):
        for j in range( M ):
            for k in range( N ):
                C[ i, j ] += A[ i, k ] * B[ k, j ]

if __name__ == "__main__":
    import numpy as np
    import time
    import math

    M = 20
    N = 20

    A = np.empty((M, N), dtype=np.double)
    B = np.empty((N, M), dtype=np.double)
    C = np.empty((M, M), dtype=np.double)

%time matrix_prod(A,B,C,N,M)

CPU times: user 9.63 ms, sys: 262 µs, total: 9.89 ms
Wall time: 9.57 ms


## Using pyccel

In [ ]:
def matrix_prod_pyc(A : 'float[:,:]', B : 'float[:,:]', C:'float[:,:]', N : 'int', M :'int'):
    
    for i in range( M ):
        for j in range( N ):
            A[ i, j ] = (i + 1) + (j + 1)

    
    for i in range( N ):
        for j in range( M ):
            B[ i, j ] = (i + 1) - (j + 1)

    
    for i in range( M ):
        for j in range( M ):
            C[ i, j ] = 0

    
    for i in range( M ):
        for j in range( M ):
            for k in range( N ):
                C[ i, j ] += A[ i, k ] * B[ k, j ]

In [ ]:
import numpy as np
import time
import math

M = 20
N = 20

A = np.empty((M, N), dtype=np.double)
B = np.empty((N, M), dtype=np.double)
C = np.empty((M, M), dtype=np.double) 

In [ ]:
matrix_prod_pyccel = epyccel(matrix_prod_pyc)
%time matrix_prod_pyccel(A,B,C,N,M)   

CPU times: user 25 µs, sys: 5 µs, total: 30 µs
Wall time: 33.6 µs


## Using OpenMP 

In [ ]:
%%file matrice_prod.py
from pyccel.stdlib.internal.openmp import omp_set_num_threads, omp_set_dynamic
from pyccel.decorators import types
from pyccel.stdlib.internal.openmp import omp_set_num_threads, omp_get_num_threads, omp_get_thread_num
import numpy as np

@types('int','int')
def pro_mat_collaps(M,N):
    #omp_set_dynamic(0)
    #omp_set_num_threads(int(n))
    
    print ("N = ", N,"  M = ", M,"\n" )
    
    A = np.empty((M, N))
    B = np.empty((N, M))
    C = np.empty((M, M))
    
    
    #$ omp parallel
    #$ omp for collapse(2)
    for i in range(M):
        for j in range(N):
            A[ i, j ] = (i + 1) + (j + 1)
    #$ omp end parallel
    

    #$ omp parallel
    #$ omp for collapse(2)
    for i in range(N):
        for j in range(M):
            B[ i, j ] = (i + 1) - (j + 1)
    #$ omp end parallel

    #$ omp parallel
    #$ omp for collapse(2)
    for i in range( M ):
        for j in range( M ):
            C[ i, j ] = 0
    #$ omp end parallel

    
    #$ omp parallel
    #$ omp for collapse (3)
    for i in range( M ):
        for j in range( M ):
            for k in range( N ):
                C[ i, j ] += A[ i, k ] * B[ k, j ]
    result = omp_get_num_threads()
    #$ omp end parallel
   
    return result

if __name__ == "__main__" :

    M = 20
    N = 20
    x = pro_mat_collaps(M,N)
    print("Execution of Matrix production in parallele with  ",x , "threads\n\n")

Overwriting matrice_prod.py


In [ ]:
!pyccel matrice_prod.py --language=c --openmp

In [ ]:
%time !export OMP_NUM_THREADS=16; ./matrice_prod

N =  20   M =  20 

Execution of Matrix production in parallele with   16 threads


CPU times: user 4.27 ms, sys: 4.13 ms, total: 8.39 ms
Wall time: 116 ms


## Exercise 3 Jacobi method

In [ ]:
import numpy as np
import math,time
import random, sys
@types('float [:,:]','float [:]','float [:]','int','float')
def jacobi(a ,b, x, n, eps):
    x_courant = np.empty(n, dtype=np.double)
    iteration = 0

    #$ omp parallel
    #pragma omp sections nowait
    while(1):
        
        iteration += 1
        
        for i in range(n):
            x_courant[i] = 0
    #pragma omp section
            for j in range(i):
                x_courant[i] += a[ j, i ] * x[j]
    #pragma omp end section
    
    #pragma omp section
            for j in range(i + 1, n):
                x_courant[i] += a[ j, i ] * x[j]
    #pragma omp end section
            x_courant[i] = (b[i] - x_courant[i]) / a[ i, i ]

        # Convergence test
        
        absmax = 0.
        for i in range(n):
            curr = abs(x[i] - x_courant[i])
            if curr > absmax:
                absmax = curr
        stand = absmax / n
        if stand <= eps or iteration > n:
            break

        # copy x_courant into x
        for i in range(n):
            x [i] = x_courant[i]
    
    #pragma omp end sections
    #$ omp end parallel    
            
    return stand, iteration


if __name__ == "__main__":
    n = 20
    diag = 20

    np.random.seed(421)  # for reproductibles results
    a = np.random.uniform(0, 1, size=(n, n))
    b = np.random.uniform(0, 1, size=n)

    # We strengthen the main diagonal of the matrix
    for i in range(n):
        a[ i, i ] += diag

    # Initial solution
    x = np.ones(n, dtype=np.double)

    # start CPU timing
    cpu_0 = time.process_time()

    # start Wall timing
    elapsed_0 = time.time()

    # Pyccelize jacobi function
    from pyccel.epyccel import epyccel
    p_jacobi = epyccel(jacobi, language='c', accelerators=['openmp'])

    eps = sys.float_info.epsilon

    # -------------------- Start computing ------------------
    stand, iteration = p_jacobi(a, b, x, n, eps)
    # ------------------ End computing ----------------

    # CPU time
    cpu_1 = time.process_time()
    cpu = cpu_1 - cpu_0

    # Wall timing
    elapsed_1 = time.time()
    elapsed = elapsed_1 - elapsed_0

    print("System size    :", n, '\n',"Iterations     :", iteration,  '\n',"Stand          :", stand,      '\n',"Elapsed Time   :", elapsed,    '\n',"CPU time       :", cpu,        '\n')

System size    : 20 
 Iterations     : 22 
 Stand          : 0.000600382935325167 
 Elapsed Time   : 0.6759302616119385 
 CPU time       : 0.0587195429999996 



## Exercise 4 Calculation of π



In [ ]:
%%file pi4.py
from pyccel.decorators import types


def pi(n:'int'):
    #$ omp parallel
    h = float(1/n)
    for k in range(100):
        Pi_calc = 0.
        for i in range(n):
            x = h * (i + 0.5)
            Pi_calc += 4.0 / (1. + x * x)

        # omp atomic
        Pi_calc = h * Pi_calc
        
    #omp end parallel
    
    return Pi_calc

Overwriting pi4.py


In [ ]:
from pi4 import *
print(pi(100))

3.1416009869231254


In [ ]:
import time, math
from pyccel.decorators import types
from pyccel.epyccel import epyccel
@types('int')
def pi_1(n):
    
    h = float(1/n)
    #$ omp parallel
    for k in range (100):
        Pi_calc = 0.
        for i in range (n):
            x = h * (i + 0.5)
            Pi_calc += 4.0 / (1. + x * x)
        #$ omp atomic
        Pi_calc = h * Pi_calc
    #$ omp end parallel
    
    return Pi_calc


if __name__ == '__main__':

    n = 300

    # Length of the integration interval
    h = float(1 / n)

    from pyccel.epyccel import epyccel

    pyc_pi = epyccel(pi_1, language='c', accelerators=['openmp'])

    # start timing
    tcpu_0 = time.process_time() # CPU time
    t_elapsed_0 = time.time()    # Wall time
     #Start computing

    Pi_calc = pyc_pi(n)

    # End computing
    # CPU time spent
    tcpu_1 = time.process_time()
    tcpu = tcpu_1 - tcpu_0

    # Wall time spent
    t_elapsed_1 = time.time()
    t_elapsed = t_elapsed_1 - t_elapsed_0

    # deviation between the estimated value and the calculated value of Pi
    Pi_estime = math.acos(-1)
    deviat = math.fabs(Pi_estime - Pi_calc)

    # Print result
    print("Number of intervals        :", n,"\n","Pi_estime - Pi_calcule  :", deviat,"\n","Temps elapsed              :", t_elapsed, "\n","Temps CPU                  :", tcpu,)

   Number of intervals        : 300 
    | Pi_estime - Pi_calcule | : 9.259259248217688e-07 
    Temps elapsed              : 0.0002646446228027344 
    Temps CPU                  : 0.0002705879999993499 



## Exercise 5 Reduction of an array


In [ ]:
%%file initial.py

def initialize_table(tab, nmolec, n, nmol):
    for k in range(nmolec):
        for j in range(n):
            for i in range(nmol):
                tab[ i,j,k ] = i + j + k

Writing initial.py


In [ ]:
%%file reduction_tab1.py

from pyccel.stdlib.internal.openmp import omp_get_thread_num
from pyccel.decorators import types




def reduction(tab:'float [:,:]', tab1 : 'float [:]', tab2 : 'float [:]', nmolec :'int', nmol:'int', n:'int'):
    for k in range(nmolec):
        tab1[:nmol] = 0
        for j in range(n):
            for i in range(nmol):
                tab1[i] = tab1[i] + tab[i,j]


Overwriting reduction_tab1.py


In [ ]:
from os import error

if __name__ == '__main__':
    import numpy as np
    import time

    # default dimension
    nmolec = 100
    nmol   = 100
    n      = 10

    tab     = np.empty((nmol, n, nmolec), dtype=np.double)
    tab1    = np.empty((nmol), dtype=np.double)
    tab2    = np.zeros((nmol), dtype=np.double)

    # pyccelize reduction tab module
    from pyccel.epyccel import epyccel
    from  reduction_tab1 import reduction
    from initial import initialize_table
    
    reduction_p = epyccel(reduction, language='c', accelerators=['openmp'])

    initialize_table(tab, nmol, n, nmolec)

    # start timing
    tcpu_0 = time.process_time() 
    t_elapsed_0 = time.time()    

    # Start computing
    reduction(tab, tab1, tab2, nmolec, nmol, n)
    # End computing

    # CPU time spent
    tcpu_1 = time.process_time()
    tcpu = tcpu_1 - tcpu_0

    # Wall time spent
    t_elapsed_1 = time.time()
    t_elapsed = t_elapsed_1 - t_elapsed_0

    # verification of results
    tab2c = np.zeros((nmol), dtype=np.double)
    for k in range(nmolec):
        tab1c = np.zeros((nmol), dtype=np.double)
        for j in range(n):
            for i in range(nmol):
                tab1c[i] = tab1c[i] + tab[i, j, k]
        tab2c [:] = tab2c[:nmol] + 2 * tab1c[:]
    err = np.max(np.abs(tab2c - tab2) / np.abs(tab2c))
    print("Temps elapsed  :", t_elapsed   , "\n","Temps CPU      :", tcpu        , "\n","Erreur relative:", err         , "\n")

Temps elapsed  : 0.036789894104003906 
 Temps CPU      : 0.03671308700000031 
 Erreur relative: 0.0 



In [ ]:
import reduction_tab1
type(reduction)

function

# Exercise 6 Matrix product by the Strassen algorithm

In [ ]:
20 & 1

0

In [ ]:
import time, math
import numpy as np
from pyccel.epyccel import epyccel
from pyccel.decorators import types
from pyccel.stdlib.internal.openmp import omp_set_num_threads, omp_get_num_threads, omp_get_thread_num


def strassen_multiply(A: 'float[:,:]', B :  'float[:,:]', C :  'float[:,:]', n:'int'):

    
    import numpy as np
    
    
    if n & 1 != 0  or n < 12:
        C[:] = np.matmul(A, B)
    else:
        n2 = n // 2
        
       
        A11 = A[ 0:n2, 0:n2 ]
        A21 = A[ n2:n, 0:n2 ]
        A12 = A[ 0:n2, n2:n ]
        A22 = A[ n2:n, n2:n ]
        B11 = B[ 0:n2, 0:n2 ]
        B21 = B[ n2:n, 0:n2 ]
        B12 = B[ 0:n2, n2:n ]
        B22 = B[ n2:n, n2:n ]
        
        
        Q1 = np.empty((n2, n2), dtype=np.double)
        Q2 = np.empty((n2, n2), dtype=np.double)
        Q3 = np.empty((n2, n2), dtype=np.double)
        Q4 = np.empty((n2, n2), dtype=np.double)
        Q5 = np.empty((n2, n2), dtype=np.double)
        Q6 = np.empty((n2, n2), dtype=np.double)
        Q7 = np.empty((n2, n2), dtype=np.double)
        

        #$ omp parallel
            
        #$ omp single    
        strassen_multiply(A11+A22, B11+B22, Q1, n2)
        #$ omp end single  
        
        #$ omp single  
        strassen_multiply(A21+A22, B11, Q2, n2)
        #$ omp end single 
        #$ omp single  
        strassen_multiply(A11, B12-B22, Q3, n2)
        #$ omp end single 
        #$ omp single  
        strassen_multiply(A22, -B11+B21, Q4, n2)
        #$ omp end single 
        #$ omp single  
        strassen_multiply(A11+A12, B22, Q5, n2)
        #$ omp end single 
        #$ omp single  
        strassen_multiply(-A11+A21, B11+B12, Q6, n2)
        #$ omp end single 
        #$ omp single  
        strassen_multiply(A12-A22, B21+B22, Q7, n2)
        
        #$ omp end single 
        #res = omp_get_num_threads()
        #$ omp end parallel
        
        C[   :n2,   :n2 ] = Q1+Q4-Q5+Q7
        C[ n2: n,   :n2 ] = Q2+Q4
        C[   :n2, n2:n  ] = Q3+Q5
        C[ n2:n , n2:n  ] = Q1+Q3-Q2+Q6
        
        
    


def calcul_erreur(n, A, B, C):
    D = np.matmul(A, B) 
    error = 0.0
    for i in range(n):
        for j in range(n):
            e = C[ i, j ] - D[ i, j ]
            error += e * e

    error = math.sqrt(error) / n
    return error

if __name__ == '__main__':
    n = 50

    # Initialize Matrices
    np.random.seed(421)
    A = np.random.uniform(0, 1, (n, n))
    B = np.random.uniform(0, 1, (n, n))
    C = np.empty((n,n), dtype=np.double)

    from pyccel.epyccel import epyccel
    
    strassen = epyccel(strassen_multiply, accelerators=['openmp'])
    #num_threads = epyccel(threads_num, accelerators=['openmp'])
    #print("Execution of Strassen in parallele with", num_threads(), "threads")
    
    
    # Start timing
    tcpu_0 = time.process_time() # CPU time
    t_elapsed_0 = time.time()    # Wall time
    # -------------------- computing ------------------
    # Calcul C=A*B par la methode recursive de Strassen
    strassen(A, B, C, n)
    # ------------------ end computing ----------------
    # CPU time spent
    tcpu_1 = time.process_time()
    tcpu = tcpu_1 - tcpu_0

    # Wall time spent
    t_elapsed_1 = time.time()
    t_elapsed = t_elapsed_1 - t_elapsed_0
    print("   Erreur        :", calcul_erreur(n, A, B, C),  "\n","   Temps elapsed :", t_elapsed,"\n","   Temps CPU     :", tcpu,"\n",)

   Erreur        : 4.117930793327016e-15 
    Temps elapsed : 0.0004563331604003906 
    Temps CPU     : 0.0012285329999990324 

